In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformer_lens
!pip uninstall jax jaxlib -y
!pip install jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-h

In [2]:
from transformer_lens import HookedTransformer
import torch

import json
import time

from src.neuron_heads import head_attribution_over_all_data
from src.datahandlers import ActivatingDataset
from src.utils import tuple_str_to_tuple

In [3]:
# Load jsons from ../experiment_data/1_next_token_neurons
# filename = "2023-10-12_10-51-10_gpt2-large"
# filename = "2023-10-13_07-03-35_gpt2-small_test"
# filename = "2023-10-13_07-54-07_gpt2-medium_test"
# filename = "2023-10-13_12-36-52_gpt2-large_test"
# filename = "2023-10-14_08-18-02_gpt2-large_mid"
filename = "2023-10-14_18-20-09_gpt2-large_mid_test"

with open(f'./experiment_data/3_trimmed_texts/{filename}.json') as f:
    text_metadata = json.load(f)

neurons = [tuple_str_to_tuple(x) for x in text_metadata['neuron_to_trunc_data'].keys()]

In [4]:
!ls experiment_data/3_trimmed_texts

2023-10-12_06-52-17_gpt2-small.json
2023-10-12_07-15-42_gpt2-medium.json
2023-10-12_10-51-10_gpt2-large.json
2023-10-12_10-51-25_gpt2-large.json
2023-10-13_07-03-35_gpt2-small_test.json
2023-10-13_07-54-07_gpt2-medium_test.json
2023-10-13_12-36-52_gpt2-large_test.json
2023-10-14_08-18-02_gpt2-large_mid.json
2023-10-14_18-20-09_gpt2-large_mid_test.json


In [5]:
# from datasets import load_dataset
# dataset = load_dataset("NeelNanda/pile-10k", split="train")
# dataset_text_list = [x['text'] for x in dataset]


import pickle

with open('./text_list_dict_test.pkl', 'rb') as f:
    dataset_text_dict = pickle.load(f)

dataset_text_list = [x['text'] for x in dataset_text_dict]
dataset = dataset_text_dict

data = ActivatingDataset(text_metadata['neuron_to_trunc_data'], dataset)
data.remove_prompts_longer_than(100)

In [6]:
# Parameters
parameters = text_metadata['parameters']
model_name = parameters['model_name']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device=device,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


In [8]:
head_attribution_dict = head_attribution_over_all_data(
    model=model,
    data=data,
    device=device,
    neurons=neurons,
    batch_size=2,
)

  0%|          | 0/100 [00:00<?, ?it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results 

  1%|          | 1/100 [00:03<05:40,  3.44s/it]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results 

  2%|▏         | 2/100 [00:06<05:12,  3.19s/it]

...

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results 

 99%|█████████▉| 99/100 [04:22<00:02,  2.51s/it]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results 

100%|██████████| 100/100 [04:25<00:00,  2.65s/it]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


In [9]:
output = {
    'parameters': parameters,
    'head_attributions': head_attribution_dict,
    'prior_filename': filename,
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_mid_test.json"

with open(f'./experiment_data/4_head_attributions/{new_filename}', 'w') as f:
    json.dump(output, f)

In [5]:
import json

with open("../experiment_data/4_head_attributions/2023-10-12_14-43-12_gpt2-small.json") as f:
    output1 = json.load(f)

with open("../experiment_data/4_head_attributions/2023-10-12_22-00-53_gpt2-small.json") as f:
    output2 = json.load(f)

# Check if two dictionaries are the same throughout
output1 == output2

False